<a href="https://colab.research.google.com/github/soccy/Study/blob/main/hongong_03_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특성 공학과 규제: 여러 특성을 사용사용한 다중 회귀에 대해 배우고, 사이킷런의 여러 도구를 사용해 보자
# 복잡한 모델의 과대적합을 막기 위한 릿지(Ridge)와 라쏘(Lasso) 회귀를 배워보자

